# ![m'lady](https://repository-images.githubusercontent.com/64991887/dc855780-e34b-11ea-9ab8-e08ca33288b0)

# <span style="color:black">***LightAutoML for Stacking***</span>
#### <span style="color:black">Hellow guys, here I am providing you with my second stage solution (staking). Hope you will enjoy it </span>.
#### The main code is provided by [LightAutoML HW_1 with custom FE](https://www.kaggle.com/alexryzhkov/lightautoml-hw-1-with-custom-fe)

##### * The transformed data is mine. 
##### * It is the result from the prediction five models (LightGBM, XGBoost, CatBoost, Random Forest, and Logistic Regression). 
##### * For tree-based models, the data was left as is because they are already in the form of "*Label Encoded*" Data, while one-hot encoding was used for Logistic Regression as it expects the data to be normalized.

# ================

In [ ]:
# install Light Auto ML
!pip install -U lightautoml

In [ ]:
# Standard python libraries
import logging
import os
import time
import requests
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task


In [ ]:
# Read the transformed test data
test_data = pd.read_csv('../input/new-with-calibrated/test_5_new_models.csv')
# Read the transformed training data
train_data = pd.read_csv('../input/new-with-calibrated/X_5_new_models.csv')
train_data['target'] = pd.read_csv('../input/new-with-calibrated/y.csv')
# Read the submission data
submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
# Define the run attributes
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 3600 # Time in seconds for automl run - UPDATED VALUE FOR UTILIZATION
TARGET_NAME = 'target' # Target column name

In [ ]:
%%time
# COMPETITION METRIC IS MAE SO WE SET IT FOR OUR TASK
task = Task('multiclass', metric='crossentropy')

In [ ]:
%%time

roles = {'target': TARGET_NAME}

In [ ]:
%%time 
# CHANGED TabularAutoML to TabularUtilizedAutoML for timeout utilization
automl = TabularUtilizedAutoML(task = task, 
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                      )
oof_pred = automl.fit_predict(train_data, roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

In [ ]:
%%time

# Fast feature importances calculation
fast_fi = automl.get_feature_scores('fast')
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (20, 10), grid = True)

In [ ]:
%%time

test_pred = automl.predict(test_data)
logging.info('Prediction for test data:\n{}\nShape = {}'
              .format(test_pred, test_pred.shape))

logging.info('Check scores...')
logging.info('OOF score: {}'.format(mean_absolute_error(train_data[TARGET_NAME].values, oof_pred.data[:, 0])))

In [ ]:
submission.iloc[:, 1:] = test_pred.data
submission.to_csv('stacking_automl_5_with_calibrated_models.csv', index=False)

# *Good Luck!*

# ================